<a href="https://colab.research.google.com/github/Napawan2005/NLP-517432/blob/main/LM_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**เหมือนกันตรงที่**
- ใช้แนวคิด transformer deep learning [1] เดียวกันทั้งหมด
- มีรูปแบบการตอบแบบ **text**[2]

**ข้อแตกต่าง**
- ตัวโมเดลใหญ่ขึ้น ข้อมูลถูกเทรนมากขึ้น

  - GPT-2 (2019) ข้อมูลถูกเทรน 1.5 พันล้าน

  - GPT-3(2020) ข้อมูลถูกเทรน 175 พันล้าน

  - GPT-4(2020) (ไม่เปิดเผย) คาดว่าระดับล้านล้าน


- มีจำนวนชั้น NN
  - GPT-2 (2019) = 48 Layers

  - GPT-3(2020) = 175 Layer

  - GPT-4(2020) (ไม่เปิดเผย) คาดว่า มากกว่า 175 layer

- Training Data (Amount and Sources)

  - GPT-2 (2019) = 40 GB of text from a variety of sources, including Wikipedia, web pages, and books
 - GPT-3(2020) = 570 GB of text from a variety of sources, including books, scientific papers, and web pages
 - GPT-4(2020) = Unknown, but likely even more diverse sources of text[2]



 อ้างอิง :
 - [1][www.ibm.com](https://www.ibm.com/think/topics/gpt)
 - [2][https://iq.opengenus.org/](https://iq.opengenus.org/gpt2-vs-gpt3-vs-gpt35-vs-gpt4/)




### ทดลองทำกับภาษาไทย

โดยใช้ชุดข้อมูลภาษาไทย เช่น PyThaiNLP สำหรับ Tokenization และนำมาสร้าง n-grams

In [31]:
from datasets import load_dataset

# 1. โหลด Dataset
dataset = load_dataset("wisesight_sentiment")

# 2. ดูโครงสร้างข้อมูล
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['texts', 'category'],
        num_rows: 21628
    })
    validation: Dataset({
        features: ['texts', 'category'],
        num_rows: 2404
    })
    test: Dataset({
        features: ['texts', 'category'],
        num_rows: 2671
    })
})


In [32]:
print(dataset['train'][0])

{'texts': 'ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกรานต์เลย รอขับอยู่นาจา กระทัดรัด เหมาะกับสาวๆขับรถคนเดียวแบบเรา ราคาสบายกระเป๋า ประหยัดน้ำมัน วิ่งไกลแค่ไหนหายห่วงค่ะ', 'category': 1}


In [33]:
import pandas as pd
df = pd.DataFrame(dataset['train'])
df

,texts,category
0,ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกร...,1
1,เปิดศักราชใหม่! นายกฯ แถลงข่าวก่อนการแข่งขันศึ...,1
2,บัตรสมาชิกลดได้อีกไหมคับ,1
3,สนใจ new mazda2ครับ,1
4,😍😍,0
...,...,...
21623,ไม่ค่อยอยากกินเล๊ย💘,0
21624,คิดถึงแม่รุ้งอีกแล้ว,0
21625,วันนี้อะไปลองมาละบลัช 4u2 สีที่จะเอาหมดอีก โอย...,2
21626,ตัวอยู่พฤกษาใจไปแสนสิริ5555555,1


In [34]:
import re
import pythainlp
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

# Load Thai stopwords once
stop_words = set(thai_stopwords())

def preprocess_text(text):


    # 1. Normalization: จัดการสระซ้อนและวรรณยุกต์ที่วางตำแหน่งผิดด้วย PyThaiNLP
    text = pythainlp.util.normalize(text)


    # 2. Remove URLs: ลบลิงก์ที่ขึ้นต้นด้วย http, https หรือ www
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # 3. Remove Special Characters: ลบสัญลักษณ์พิเศษ, Emoji และเครื่องหมายคำพูด
    #    เก็บเฉพาะตัวอักษรไทย อังกฤษ และตัวเลข
    text = re.sub(r'[^\u0E00-\u0E7Fa-zA-Z0-9\s]', '', text)

    # 4. Tokenization: ตัดข้อความยาวๆ ให้เป็น List ของคำ (Tokens)
    #    ใช้ engine='newmm' ซึ่งเป็นมาตรฐานสูงสุดของภาษาไทย
    tokens = word_tokenize(text, engine='newmm')

    # 5. Remove Stopwords & Short words:
    #    - ลบคำที่อยู่ในรายการ stop_words
    #    - ลบคำที่มีความยาวน้อยกว่าหรือเท่ากับ 1 ตัวอักษร (เช่น 'ก', 'ที่' บางกรณี)
    #    - ลบช่องว่าง (Whitespace)
    tokens = [t for t in tokens if t not in stop_words and len(t) > 1 and not t.isspace()]

    return tokens

print("The 'preprocess_text' function has been defined and Thai stopwords have been loaded.")

The 'preprocess_text' function has been defined and Thai stopwords have been loaded.


In [35]:
df['tokens'] = df['texts'].apply(preprocess_text)

In [36]:
df

,texts,category,tokens
0,ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกร...,1,"[จอง, แล้, วนา, จา, Mitsubishi, Attrage, สงกรา..."
1,เปิดศักราชใหม่! นายกฯ แถลงข่าวก่อนการแข่งขันศึ...,1,"[ศักราช, นายกฯ, แถลงข่าว, การแข่งขัน, ศึก, ช้า..."
2,บัตรสมาชิกลดได้อีกไหมคับ,1,"[บัตรสมาชิก, ลด, ไหม, คับ]"
3,สนใจ new mazda2ครับ,1,"[สนใจ, new, mazda]"
4,😍😍,0,[]
...,...,...,...
21623,ไม่ค่อยอยากกินเล๊ย💘,0,"[กิน, เล๊ย]"
21624,คิดถึงแม่รุ้งอีกแล้ว,0,"[คิดถึง, แม่, รุ้ง, อีกแล้ว]"
21625,วันนี้อะไปลองมาละบลัช 4u2 สีที่จะเอาหมดอีก โอย...,2,"[อะ, ลอง, บลัช, สี, ที่จะ, โอย, ชุ้นคง, บุญ]"
21626,ตัวอยู่พฤกษาใจไปแสนสิริ5555555,1,"[ตัว, พฤกษา, ใจ, แสน, สิริ, 5555555]"


In [37]:
all_tokens = []
for tokens_list in df['tokens']:
    all_tokens.extend(tokens_list)

print(f"Total number of tokens: {len(all_tokens)}")

Total number of tokens: 253151


In [38]:
bigrams = list(zip(all_tokens, all_tokens[1:]))

print(f"Total number of bigrams: {len(bigrams)}")

Total number of bigrams: 253150


In [39]:
from collections import defaultdict, Counter

# สร้าง Dictionary เก็บคำถัดไปที่อาจเกิดขึ้นได้
# key = คำปัจจุบัน, value = รายการคำที่เคยตามหลังคำนั้น
prediction_dict = defaultdict(list)

for w1, w2 in bigrams:
    prediction_dict[w1].append(w2)

In [40]:
# ทำสรุปให้แต่ละคำว่า คำที่ตามหลังมา คำไหนฮิตที่สุด
for word in prediction_dict:
    prediction_dict[word] = Counter(prediction_dict[word]).most_common(1)

In [41]:
def predict_next_word(text):
    # 1. ตัดคำที่รับเข้ามา
    tokens = word_tokenize(text, engine="newmm")
    if not tokens:
        return "กรุณาพิมพ์ข้อความ"

    # 2. เอาคำสุดท้ายไปค้นหา
    last_word = tokens[-1]

    if last_word in prediction_dict:
        # ดึงคำที่พบบ่อยที่สุดออกมา
        next_word = prediction_dict[last_word][0][0]
        return next_word
    else:
        return "ขออภัย ไม่พบข้อมูลคำถัดไป"

# ตัวอย่างการใช้งาน
input_text = "อาหาร"
next_word = predict_next_word(input_text)
print(f"คำถัดไปที่น่าจะเป็น: {input_text}{next_word}")

คำถัดไปที่น่าจะเป็น: อาหารอร่อย


In [42]:
def generate_sentence(start_word, length=5):
    current_word = start_word
    sentence = [start_word]

    for _ in range(length):
        next_w = predict_next_word(current_word)
        if "ไม่พบข้อมูล" in next_w:
            break
        sentence.append(next_w)
        current_word = next_w

    return "".join(sentence)

# ลองรันดู
print(generate_sentence("อาหาร"))

อาหารอร่อยกินเบียร์ช้างขวด
